In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("Motor_Vehicle_Collisions_-_Crashes.csv")
df.head()


In [ ]:
df.to_json('collisions_raw.json', orient='records', lines=True)
df.to_parquet('collisions_raw.parquet')

In [ ]:


# dtypes: float64(4), int64(7), object(18)

# df.info()
object_columns = df.select_dtypes(include='int64').columns
print(object_columns)

In [ ]:
object_columns = df.select_dtypes(include='object').columns
print(object_columns)

In [ ]:
object_columns = df.select_dtypes(include='float64').columns
print(object_columns)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()


In [ ]:
df = df.dropna(subset=["BOROUGH", "LATITUDE", "LONGITUDE", "LOCATION"])


In [ ]:
numeric_cols = ["NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"]
df[numeric_cols] = df[numeric_cols].fillna(0)

In [ ]:
catstrig_cols=[ 'ZIP CODE','ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5']
df[catstrig_cols]=df[catstrig_cols].fillna("Unknown")

In [ ]:
df.isnull().sum() 


In [ ]:
#convert date time datatype
Datime=df['CRASH DATE'] + ' ' + df['CRASH TIME']
df['Full_Date']=pd.to_datetime(Datime)

In [ ]:
#convert floattype to int64
df['NUMBER OF PERSONS INJURED']=df['NUMBER OF PERSONS INJURED'].astype('Int64')
df['NUMBER OF PERSONS KILLED']=df['NUMBER OF PERSONS KILLED'].astype('Int64')

In [ ]:
df['BOROUGH'] = df['BOROUGH'].astype('category')
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].astype('category')
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].astype('category')
df['CONTRIBUTING FACTOR VEHICLE 3']=df['CONTRIBUTING FACTOR VEHICLE 3'].astype('category')
df['CONTRIBUTING FACTOR VEHICLE 4']=df['CONTRIBUTING FACTOR VEHICLE 4'].astype('category')
df['CONTRIBUTING FACTOR VEHICLE 5']=df['CONTRIBUTING FACTOR VEHICLE 5'].astype('category')

In [ ]:
factor_cols = [
    'CONTRIBUTING FACTOR VEHICLE 1',
    'CONTRIBUTING FACTOR VEHICLE 2',
    'CONTRIBUTING FACTOR VEHICLE 3',
    'CONTRIBUTING FACTOR VEHICLE 4',
    'CONTRIBUTING FACTOR VEHICLE 5'
]

for col in factor_cols:
    df[col] = df[col].astype('category')
    

In [ ]:
print("befor cleaning")
print(df[['BOROUGH', 'CONTRIBUTING FACTOR VEHICLE 1']].sample(5, random_state=1))

In [ ]:



text_cols = df.select_dtypes(include=['object', 'category']).columns

for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)          
        .str.strip()          
        .str.title()          
        .replace({'0': np.nan, 'Unspecified': np.nan, 'Nan': np.nan})
    )


In [ ]:
print("after cleaning")
print(df[['BOROUGH', 'CONTRIBUTING FACTOR VEHICLE 1']].sample(5, random_state=1))

In [ ]:
# #Examples for Filtering to data :
# df2019=df[df['Full_Date']==]

In [ ]:
# Step 1: Filter data by a specific date range
# For example, let's select collisions between 2021-01-01 and 2021-12-31
start_date = '2021-01-01'
end_date = '2021-12-31'
df_period = df[(df['Full_Date'] >= start_date) & (df['Full_Date'] <= end_date)]

# Step 2: Filter data by borough
df_manhattan = df_period[df_period['BOROUGH'] == 'Manhattan']

# Step 3: Check the size of the filtered dataset
print("Filtered dataset shape:", df_manhattan.shape)

# Step 4: Group by BOROUGH and compute total number of injured, sorted descending
borough_injured = df_period.groupby('BOROUGH')['NUMBER OF PERSONS INJURED'].sum().sort_values(ascending=False)
print(borough_injured)

# Step 5: Extract year and month from Full_Date for time-based analysis
df_period.loc[:, 'Year'] = df_period['Full_Date'].dt.year
df_period.loc[:, 'Month'] = df_period['Full_Date'].dt.month

# Step 6: Group by year and borough to count number of collisions per year per borough, sorted

collisions_per_year = df_period.groupby(['Year', 'BOROUGH']).size().sort_values(ascending=False)
print(collisions_per_year)

# Step 7: Pivot table sorted by columns (borough alphabetically)
pivot_table = df_period.pivot_table(
    index='Year',
    columns='BOROUGH',
    values='COLLISION_ID',  
    aggfunc='count'
).sort_index(axis=1)  # sort borough columns alphabetically
print(pivot_table)

# Step 8: Correlation matrix, rounded and displayed nicely
correlation_matrix = df_period[numeric_cols].corr().round(2)
print(correlation_matrix)
#***************i add more optional things to code ************************

In [ ]:

numeric_cols = [
    'NUMBER OF PERSONS INJURED',
    'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED',
    'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED',
    'NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED',
    'NUMBER OF MOTORIST KILLED'
]


numeric_summary = df[numeric_cols].describe()
print(numeric_summary)

medians = df[numeric_cols].median()
print("Medians:\n", medians)


In [ ]:
df.to_json('collisions_clean.json', orient='records', lines=True)
df.to_parquet('collisions_clean.parquet')